# Logic converting bounding boxes around words to bounding boxes around the full sentence  

## Steps

1) Put all of our word bounding box data into a pandas DataFrame() structure (csv)
2) Look at the top left pixel of the first word in a sentence and add the width to get to the top right pixel of the word
3) Check the distance of the top right pixel from the top left pixel of the word directly to its right.
       -If it is > threshold value, then the sentence bounding box should be completed
       -If it is < threshold value, then the two word bounding boxes should be merged


   




In [4]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import pandas as pd
from PIL import Image
import PyPDF2
import pytesseract
from pytesseract import Output
from pdf2image import convert_from_path

## Download pytesseract instructions

Download Tesseract on your computer - https://sourceforge.net/projects/tesseract-ocr.mirror/files/5.5.0/tesseract-ocr-w64-setup-5.5.0.20241111.exe/download 
- make sure to add C:\Program Files\Tesseract-OCR to your system PATH

Paste this into your terminal to verify intallation worked: tesseract --version

Install python wrapper by pasting this into your terminal: pip install pytesseract Pillow pandas


## Turn pdf into image 

In [22]:
#pdf_path = r"C:\Users\dkhun\UC Davis\AISC Github repository\BeginnerProjectFallQuarter2025\data\raw_pdfs\textbook_pdf_6.pdf"
pdf_path = r"C:\Users\Conno\OneDrive\Desktop\textbook_pdf_6.pdf"
pdf_image = convert_from_path(pdf_path)

pages = list(pdf_image)
pages[0].show()

pages[0].save('output_page1.png','PNG') #save PIL Image to disk as a PNG so that I can use the file path in cv2.imread below

## Start by getting all of our word bounding box data into a pandas DataFrame()

In [21]:

img = cv2.imread('output_page1.png')
data = pytesseract.image_to_data(img, output_type=Output.DICT)

df = pd.DataFrame(data)
df_NeededData = df[['left','top','width','height']]
print(df.head())
print(df_NeededData.head())

   level  page_num  block_num  par_num  line_num  word_num  left  top  width  \
0      1         1          0        0         0         0     0    0   1533   
1      2         1          1        0         0         0   275  245   1005   
2      3         1          1        1         0         0   275  245   1005   
3      4         1          1        1         1         0   275  245   1005   
4      5         1          1        1         1         1   275  250    137   

   height  conf  text  
0    2132    -1        
1      48    -1        
2      48    -1        
3      48    -1        
4      36    96  AAPA  
   left  top  width  height
0     0    0   1533    2132
1   275  245   1005      48
2   275  245   1005      48
3   275  245   1005      48
4   275  250    137      36


In [ ]:
threshold = 10
if(